In [1]:
import pandas as pd 
import numpy as np

In [2]:
ipl_matches = pd.read_csv(r'Datasets\IPL\matches.csv')

In [3]:
ipl_deliveries = pd.read_csv(r'Datasets\IPL\deliveries.csv')

In [4]:
stadium_mapping = {
    'M Chinnaswamy Stadium': 'M Chinnaswamy Stadium',
    'M.Chinnaswamy Stadium': 'M Chinnaswamy Stadium',
    'M Chinnaswamy Stadium, Bengaluru': 'M Chinnaswamy Stadium',
    'Punjab Cricket Association Stadium, Mohali': 'Punjab Cricket Association Stadium',
    'Punjab Cricket Association IS Bindra Stadium': 'Punjab Cricket Association Stadium',
    'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association Stadium',
    'Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh': 'Punjab Cricket Association Stadium',
    'Feroz Shah Kotla': 'Arun Jaitley Stadium',
    'Arun Jaitley Stadium': 'Arun Jaitley Stadium',
    'Arun Jaitley Stadium, Delhi': 'Arun Jaitley Stadium',
    'Wankhede Stadium': 'Wankhede Stadium',
    'Wankhede Stadium, Mumbai': 'Wankhede Stadium',
    'Eden Gardens': 'Eden Gardens',
    'Eden Gardens, Kolkata': 'Eden Gardens',
    'Sawai Mansingh Stadium': 'Sawai Mansingh Stadium',
    'Sawai Mansingh Stadium, Jaipur': 'Sawai Mansingh Stadium',
    'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
    'Rajiv Gandhi International Stadium': 'Rajiv Gandhi International Stadium',
    'Rajiv Gandhi International Stadium, Uppal, Hyderabad': 'Rajiv Gandhi International Stadium',
    'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
    'MA Chidambaram Stadium': 'MA Chidambaram Stadium',
    'MA Chidambaram Stadium, Chepauk, Chennai': 'MA Chidambaram Stadium',
    'Dr DY Patil Sports Academy': 'Dr DY Patil Sports Academy',
    'Dr DY Patil Sports Academy, Mumbai': 'Dr DY Patil Sports Academy',
    'Himachal Pradesh Cricket Association Stadium': 'Himachal Pradesh Cricket Association Stadium',
    'Himachal Pradesh Cricket Association Stadium, Dharamsala': 'Himachal Pradesh Cricket Association Stadium',
    'Maharashtra Cricket Association Stadium': 'Maharashtra Cricket Association Stadium',
    'Maharashtra Cricket Association Stadium, Pune': 'Maharashtra Cricket Association Stadium',
    'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
    'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
    'Sardar Patel Stadium, Motera': 'Narendra Modi Stadium',
    'Narendra Modi Stadium, Ahmedabad': 'Narendra Modi Stadium',
    'Barsapara Cricket Stadium, Guwahati': 'Barsapara Cricket Stadium',
    'Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow': 'Ekana Cricket Stadium',
    'Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur': 'Mullanpur Cricket Stadium'
}

In [5]:
ipl_matches['venue'] = ipl_matches['venue'].replace(stadium_mapping)

In [6]:
year_replace_dict={
    "2007/08":"2008",
    "2009/10":"2009",
    "2020/21":"2020"
}
ipl_matches['season']=ipl_matches['season'].replace(year_replace_dict)
ipl_matches['season'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020, 2021, 2022, 2023, 2024])

In [7]:
replacement_dict = {
    "Royal Challengers Bangalore":"Royal Challengers Bengaluru",
    "Kings XI Punjab":"Punjab Kings",
    "Delhi Daredevils":"Delhi Capitals",
    "Rising Pune Supergiant":"Rising Pune Supergiants",
    "Gujarat Lions":"Gujarat Titans",
    "Deccan Chargers":"Sunrisers Hyderabad"
}
ipl_matches['team1'] = ipl_matches['team1'].replace(replacement_dict)
ipl_matches['team2'] = ipl_matches['team2'].replace(replacement_dict)
ipl_matches['toss_winner']=ipl_matches['toss_winner'].replace(replacement_dict)
ipl_matches['winner']=	ipl_matches['winner'].replace(replacement_dict)
ipl_deliveries['batting_team']=ipl_deliveries['batting_team'].replace(replacement_dict) 
ipl_deliveries['bowling_team']=ipl_deliveries['bowling_team'].replace(replacement_dict)

In [8]:
ipl_deliveries['balls_bowled']=ipl_deliveries['extras_type'].apply(lambda x: 0 if x in ['wides','noballs'] else 1)

In [9]:
ipl_deliveries['bowler_wicket']=ipl_deliveries.dismissal_kind.apply(lambda x: 1 if x in ['caught','bowled','lbw','stumped','caught and bowled','hit wicket'] else 0)
ipl_deliveries['fielder_wicket']=ipl_deliveries.dismissal_kind.apply(lambda x: 1 if x in ['caught','run out','stumped'] else 0)

In [10]:
ipl_deliveries = ipl_deliveries.sort_values(by=['match_id', 'inning', 'over', 'ball'])

# Calculate cumulative team runs and wickets at each ball
ipl_deliveries['total_team_runs'] = ipl_deliveries.groupby(['match_id', 'inning'])['total_runs'].cumsum()
ipl_deliveries['wickets_fallen'] = ipl_deliveries.groupby(['match_id', 'inning'])['is_wicket'].cumsum()

# Now, create the Fall of Wicket (FOW) column
ipl_deliveries['Fall_of_Wicket'] = ipl_deliveries.apply(
    lambda x: f"{x['total_team_runs']}/{x['wickets_fallen']}" if not pd.isna(x['player_dismissed']) else None, axis=1
)


In [11]:
df_bat_innings = ipl_deliveries.groupby(by=['match_id','batter','inning']).agg(
    Team=('batting_team', 'first'),
    Opposition=('bowling_team', 'first'),
    Over=('over','first'),
    Ball=('ball','first'),
    Runs=('batsman_runs', 'sum'),
    BF=('balls_bowled', 'sum'),
    Dots=('batsman_runs',lambda x: (x == 0).sum()),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    Out=('player_dismissed', lambda x: 0 if pd.isna(x.iloc[-1]) else 1),
    Dismissal_Kind=('dismissal_kind', lambda x: x.dropna().iloc[0] if not x.dropna().empty else None),
    Fall_of_Wicket=('Fall_of_Wicket', 'last')
).reset_index().sort_values(by=['match_id','inning','Over','Ball'])

In [12]:
df_bat_innings['Strike_rate']=round((df_bat_innings['Runs']/df_bat_innings['BF'])*100,2)

In [13]:
df_bat_innings

,match_id,batter,inning,Team,Opposition,Over,Ball,Runs,BF,Dots,Fours,Sixes,Out,Dismissal_Kind,Fall_of_Wicket,Strike_rate
12,335982,SC Ganguly,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,1,10,12,8,2,0,1,caught,61/1,83.33
2,335982,BB McCullum,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,2,158,73,23,10,13,0,None,None,216.44
10,335982,RT Ponting,1,Kolkata Knight Riders,Royal Challengers Bengaluru,5,3,20,20,9,1,1,1,caught,112/2,100.00
4,335982,DJ Hussey,1,Kolkata Knight Riders,Royal Challengers Bengaluru,13,2,12,12,4,1,0,1,caught,172/3,100.00
7,335982,Mohammad Hafeez,1,Kolkata Knight Riders,Royal Challengers Bengaluru,17,3,5,3,1,1,0,0,None,None,166.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16569,1426312,B Kumar,1,Sunrisers Hyderabad,Kolkata Knight Riders,17,6,0,1,1,0,0,0,None,None,0.00
16575,1426312,Rahmanullah Gurbaz,2,Kolkata Knight Riders,Sunrisers Hyderabad,0,1,39,32,21,5,2,1,lbw,102/2,121.88
16576,1426312,SP Narine,2,Kolkata Knight Riders,Sunrisers Hyderabad,1,1,6,2,1,0,1,1,caught,11/1,300.00
16580,1426312,VR Iyer,2,Kolkata Knight Riders,Sunrisers Hyderabad,1,3,52,26,6,4,3,0,None,None,200.00


In [14]:
df_bowl_innings = ipl_deliveries.groupby(['match_id', 'bowler','inning']).agg(
    Team=('bowling_team', 'first'),
    Opposition=('batting_team', 'first'),
    Ball_Count=('balls_bowled','sum'),  # Total balls bowled
    Over=('over','first'),
    Ball=('ball','first'),
    Runs_Conceded=('total_runs', lambda x: (ipl_deliveries.loc[x.index, 'batsman_runs'] + 
                                            ipl_deliveries.loc[x.index, 'extra_runs']
                                            .where(~ipl_deliveries.loc[x.index, 'extras_type'].isin(['byes', 'legbyes', 'penalty']), 0)).sum()),
    Wickets=('dismissal_kind', lambda x: (x[~x.isin(['run out'])].notna().sum())),  
    Dot_Balls=('total_runs', lambda x: (x == 0).sum()),
    Fours=('batsman_runs', lambda x: (x == 4).sum()), 
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    wides = ('extras_type',lambda x: (x == 'wides').sum()),
    Caught = ('dismissal_kind',lambda x: (x == 'caught').sum()),
    bowled = ('dismissal_kind',lambda x: (x == 'bowled').sum()),
    lbw = ('dismissal_kind',lambda x: (x == 'lbw').sum()),
    candb=('dismissal_kind',lambda x: (x =='caught and bowled').sum()),
    hit_wicket=('dismissal_kind',lambda x: (x =='hit wicket').sum()),
    stumped=('dismissal_kind',lambda x: (x =='stumped').sum())
).reset_index().sort_values(by=['match_id','inning','Over','Ball'])

In [15]:
df_bowl_innings['Overs'] = df_bowl_innings['Ball_Count'] // 6 + (df_bowl_innings['Ball_Count'] % 6) / 10

In [16]:
df_bowl_innings['Econ'] = round((df_bowl_innings['Runs_Conceded']/df_bowl_innings['Overs']),2)

In [17]:
df_fielders = ipl_deliveries.groupby(['match_id', 'fielder','inning']).agg(
    Team=('bowling_team', 'first'),
    Opposition=('batting_team', 'first'),
    Caught = ('dismissal_kind',lambda x: (x == 'caught').sum()),
    run_out = ('dismissal_kind',lambda x: (x =='run out').sum()),
    stumped=('dismissal_kind',lambda x: (x =='stumped').sum())
).reset_index().sort_values(by=['match_id','inning'])

In [18]:

ipl_matches['result_margin'] = pd.to_numeric(ipl_matches['result_margin'], errors='coerce').fillna(0).astype(int)
ipl_matches['target_runs'] = pd.to_numeric(ipl_matches['target_runs'], errors='coerce').fillna(0).astype(int)

In [19]:
df_teams = ipl_deliveries.groupby(['match_id','batting_team','bowling_team','inning']).agg(
    Runs=('total_runs','sum'),
    wickets_fallen=('is_wicket', 'sum'),
    Caught = ('dismissal_kind',lambda x: (x == 'caught').sum()),
    bowled = ('dismissal_kind',lambda x: (x == 'bowled').sum()),
    run_out = ('dismissal_kind',lambda x: (x =='run out').sum()),
    stumped=('dismissal_kind',lambda x: (x =='stumped').sum()),
    lbw=('dismissal_kind',lambda x: (x =='lbw').sum()),
    Candb=('dismissal_kind',lambda x: (x =='caught and bowled').sum()),
    hit_wicket = ('dismissal_kind',lambda x: (x =='hit wicket').sum()),
    Ones=('batsman_runs', lambda x: (x == 1).sum()),
    Twos=('batsman_runs', lambda x: (x == 2).sum()),
    Threes=('batsman_runs', lambda x: (x == 3).sum()),
    Fours=('batsman_runs', lambda x: (x == 4).sum()),
    Sixes=('batsman_runs', lambda x: (x == 6).sum()),
    dots=('total_runs',lambda x: (x == 0).sum()),
    Ball_Count=('balls_bowled','sum'),
    Extras = ('extra_runs','sum')
).reset_index().sort_values(by=['match_id','inning'])

In [20]:
df_teams['Overs'] = df_teams['Ball_Count'] // 6 + (df_teams['Ball_Count'] % 6) / 10

In [21]:
df_teams['Run_Rate']=round(df_teams['Runs']/df_teams['Overs'],2) 

In [22]:
df_teams['Score'] = df_teams.apply(
    lambda row: f"{row['Runs']}/{row['wickets_fallen']}" if row['wickets_fallen'] < 10 else f"{row['Runs']}", 
    axis=1
)

In [23]:
df_bat_innings.columns

Index(['match_id', 'batter', 'inning', 'Team', 'Opposition', 'Over', 'Ball',
       'Runs', 'BF', 'Dots', 'Fours', 'Sixes', 'Out', 'Dismissal_Kind',
       'Fall_of_Wicket', 'Strike_rate'],
      dtype='object')

In [24]:
df_fow=df_bat_innings[['match_id', 'batter', 'inning', 'Team', 'Opposition','Fall_of_Wicket']]

In [25]:
df_fow.dropna(inplace=True)

C:\Users\Pranoy\AppData\Local\Temp\ipykernel_12380\4201229192.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fow.dropna(inplace=True)


In [26]:
df_fow

,match_id,batter,inning,Team,Opposition,Fall_of_Wicket
12,335982,SC Ganguly,1,Kolkata Knight Riders,Royal Challengers Bengaluru,61/1
10,335982,RT Ponting,1,Kolkata Knight Riders,Royal Challengers Bengaluru,112/2
4,335982,DJ Hussey,1,Kolkata Knight Riders,Royal Challengers Bengaluru,172/3
9,335982,R Dravid,2,Royal Challengers Bengaluru,Kolkata Knight Riders,4/1
14,335982,W Jaffer,2,Royal Challengers Bengaluru,Kolkata Knight Riders,24/4
...,...,...,...,...,...,...
16567,1426312,Abdul Samad,1,Sunrisers Hyderabad,Kolkata Knight Riders,77/7
16573,1426312,PJ Cummins,1,Sunrisers Hyderabad,Kolkata Knight Riders,113/10
16571,1426312,JD Unadkat,1,Sunrisers Hyderabad,Kolkata Knight Riders,113/9
16575,1426312,Rahmanullah Gurbaz,2,Kolkata Knight Riders,Sunrisers Hyderabad,102/2


In [27]:
ipl_matches.to_csv(r'Datasets\IPL\matches.csv',index=False)
ipl_deliveries.to_csv(r'Datasets\IPL\deliveries.csv',index=False)
df_bat_innings.to_csv(r'Datasets\IPL\batting.csv',index=False)
df_bowl_innings.to_csv(r'Datasets\IPL\bowling.csv',index=False)
df_fielders.to_csv(r'Datasets\IPL\fielder.csv',index=False)
df_teams.to_csv(r'Datasets\IPL\teams.csv',index=False)

In [28]:
matches = pd.read_csv(r'Datasets\IPL\matches.csv')

In [29]:
calendar = matches[['date','id']]

In [30]:
calendar.to_csv(r'Datasets\IPL\calender.csv',index=False)

In [31]:
points = pd.read_csv(r'Datasets\IPL\points_table.csv')

In [32]:
points

,season,Team,Match,Won,Lost,N/R,Net RR,Points,Team_Full_name
0,2008,RR,14,11,3,0,0.632,22,Rajasthan Royals
1,2008,PBKS,14,10,4,0,0.509,20,Punjab Kings
2,2008,CSK,14,8,6,0,-0.192,16,Chennai Super Kings
3,2008,DC,14,7,6,1,0.342,15,Delhi Capitals
4,2008,MI,14,7,7,0,0.570,14,Mumbai Indians
...,...,...,...,...,...,...,...,...,...
141,2024,DC,14,7,7,0,-0.377,14,Delhi Capitals
142,2024,LSG,14,7,7,0,-0.667,14,Lucknow Super Giants
143,2024,GT,14,5,7,2,-1.063,12,Gujrat Titans
144,2024,PBKS,14,5,9,0,-0.353,10,Punjab Kings


In [33]:
points['Team'] = points['Team'].str.strip()

In [34]:
points.Team.unique()

array(['RR', 'PBKS', 'CSK', 'DC', 'MI', 'KKR', 'RCB', 'SRH', 'KTK', 'RPS',
       'GT', 'LSG'], dtype=object)

In [35]:
Team_full = {
    'RR':'Rajasthan Royals', 
    'PBKS':'Punjab Kings', 
    'CSK':'Chennai Super Kings', 
    'DC':'Delhi Capitals', 
    'MI':'Mumbai Indians', 
    'KKR':'Kolkata Knight Riders', 
    'RCB':'Royal Challengers Bengaluru', 
    'SRH':'Sunrisers Hyderabad', 
    'KTK':'Kochi Tuskers Kerala', 
    'RPS':'Rising Pune Supergiants',
    'GT':'Gujrat Titans', 
    'LSG':'Lucknow Super Giants'
}

In [36]:
points['Team_Full_name'] = points['Team'].replace(Team_full)

In [37]:
points

,season,Team,Match,Won,Lost,N/R,Net RR,Points,Team_Full_name
0,2008,RR,14,11,3,0,0.632,22,Rajasthan Royals
1,2008,PBKS,14,10,4,0,0.509,20,Punjab Kings
2,2008,CSK,14,8,6,0,-0.192,16,Chennai Super Kings
3,2008,DC,14,7,6,1,0.342,15,Delhi Capitals
4,2008,MI,14,7,7,0,0.570,14,Mumbai Indians
...,...,...,...,...,...,...,...,...,...
141,2024,DC,14,7,7,0,-0.377,14,Delhi Capitals
142,2024,LSG,14,7,7,0,-0.667,14,Lucknow Super Giants
143,2024,GT,14,5,7,2,-1.063,12,Gujrat Titans
144,2024,PBKS,14,5,9,0,-0.353,10,Punjab Kings


In [38]:
points.to_csv(r'Datasets\IPL\points_table.csv',index=False)

In [39]:

pages=['matches.csv','batting.csv','bowling.csv','fielder.csv','teams.csv','points_table.csv','deliveries.csv']


with pd.ExcelWriter(r'Datasets\IPL\IPL_data.xlsx', engine='xlsxwriter') as writer:
    for page in pages:
        df = pd.read_csv(r'Datasets\IPL\{}'.format(page))
        sheet_name = page.split('.')[0] 

        df.to_excel(writer, sheet_name=sheet_name, index=False)